In [5]:
#generating modulations from displacements directly (.eig file)

import numpy
import subprocess
import pathlib

filmod_name = 'mode.vasp'
fileig_name = 'Mp_e05.eig'
filscf_name = 'scf.in'
q = [0.54, 0.54, 0.0]
facs = [2, 2, 2]
phase = 0 #numpy.pi / 2
amp = 1


aut_to_a = 0.529177249
ry_to_ev = 13.605698066
ry_to_mev = ry_to_ev * 1000

filscf = open(filscf_name, 'r')
#get ntyp, nat and alat
for line in filscf:
    #get nat
    if 'nat' in line:
        nat = int(line.split()[2])
    #get ntyp
    elif 'ntyp' in line:
        ntyp = int(line.split()[2])
    #get lattice vectors and calculate reciprocal lattice vectors
    elif 'CELL_PARAMETERS' in line:
        lat = numpy.empty(shape=(3,3), dtype=float)
        for i in range(0, 3):
            lv = filscf.readline().split()
            for j in range(0, 3):
                lat[i, j] = float(lv[j])
        lat_inv = 2*numpy.pi * numpy.linalg.inv(lat)
    #get atomic specifications
    elif 'ATOMIC_SPECIES' in line:
        atname = numpy.empty(ntyp, dtype=str)
        atmass = numpy.empty(ntyp, dtype=float)
        for i in range(0, ntyp):
            line = filscf.readline().split()
            atname[i] = line[0]
            atmass[i] = float(line[1])
    #get atomic positions and atom type 
    elif 'ATOMIC_POSITIONS' in line:
        at = numpy.empty(shape=(nat,4), dtype=float)
        for i in range(0, nat):
            pos = filscf.readline().split()
            
            at[i, 0] = numpy.where(atname == pos[0])[0][0]
            for j in range(1, 4):
                at[i, j] = float(pos[j])    
filscf.close()

fileig = open(fileig_name, 'r')
for i in range(0, 2):
    fileig.readline()
    
#get q-vector    
if q == [None]:
    q = numpy.empty(3, dtype=float)
    vec = fileig.readline().split()
    for i in range(0, 3):
        q[i] = float(vec[i + 2])

else:
    q = numpy.array(q)
    fileig.readline()
    
if facs == [None]:    
    facs = numpy.empty(3, dtype=int)
    for i, xq in enumerate(q):
        if xq == 0:
            facs[i] = 1
            continue
        facs[i] = 1/xq
else:
    facs = numpy.array(facs)

lat_mod = numpy.copy(lat)
for i, fac in enumerate(facs):
    lat_mod[i] *= fac
    
fileig.readline()   

#get modulations
nmod = 3*nat
mods = numpy.empty(shape=(nmod, nat, 3), dtype=complex)
for i in range(0, nmod):
    fileig.readline()
    for j in range(0, nat):
        vec = fileig.readline().split()
        for k in range(0, 3):
            mods[i,j,k] = complex(float(vec[k*2+1]), float(vec[k*2+2]))

#generate supercell with modulation
for imod in range(0, nmod):
    nat_mod = nat
    at_mod = numpy.empty(shape=(0,4), dtype=float)
        
    for i in range(0, facs[0]):
        for j in range(0, facs[1]):
            for k in range(0, facs[2]):
                for l, iat in enumerate(at):
                    pos = numpy.copy(iat)

                    pos[1] += i
                    pos[2] += j
                    pos[3] += k
                    
                    pos[1] /= facs[0]
                    pos[2] /= facs[1]
                    pos[3] /= facs[2]
                    
                    pos_abs = numpy.dot(lat_mod, pos[1:4])
                    q_abs = numpy.dot(lat_inv, q)
                    wave = numpy.exp(1j*numpy.dot(q_abs,pos_abs)+1j*phase)
                    disp = mods[imod,l]
                    pos[1:4] += amp*(disp*wave).astype(float) / numpy.sqrt(atmass[int(pos[0])])
                
                    at_mod = numpy.vstack((at_mod, pos))
    #sort atoms
    sortby = at_mod[:,0].argsort()
    at_mod = at_mod[sortby[::1]]
       
    #write vasp file
    filmod = open(filmod_name + str(imod), 'w')
    filmod.write('\n1.0\n')
    for lv in lat_mod:
        for num in lv:
            filmod.write('\t' + str(num) + '\t')
        filmod.write('\n')
        
    nat_typ = numpy.bincount(at_mod[:,0].astype(int))
    for iname in atname:
        filmod.write(iname + '\t')
    filmod.write('\n')
    for i, n in enumerate(nat_typ):
        filmod.write(str(n) + '\t')
    filmod.write('\nDirect\n')
    for iat in at_mod:
        for i, num in enumerate(iat):
            if i == 0:
                continue
            filmod.write(str(num) + '\t')
        filmod.write('\n')
    filmod.close()



C:\Users\noema\Anaconda3\lib\site-packages\ipykernel_launcher.py:120: ComplexWarning: Casting complex values to real discards the imaginary part


In [69]:
#generating modulations from ith dynmat in dyn file containing multiple dynmats

import numpy
import scipy.linalg

filmod_name = 'mode.vasp'
filscf_name = 'scf.in'
dynmat_filename = '../DYNMAT/Ms.dyn'

q = None
facs = [2, 2, 2]
idyn = 1
phase = numpy.pi/2
amp = -1.0

aut_to_a = 0.529177249
ry_to_ev = 13.605698066
ry_to_mev = ry_to_ev * 1000


filscf = open(filscf_name, 'r')
#get ntyp, nat and alat
for line in filscf:
    #get nat
    if 'nat' in line:
        nat = int(line.split()[2])
    #get ntyp
    elif 'ntyp' in line:
        ntyp = int(line.split()[2])
    #get lattice vectors and calculate reciprocal lattice vectors
    elif 'CELL_PARAMETERS' in line:
        lat = numpy.empty(shape=(3,3), dtype=float)
        for i in range(0, 3):
            lv = filscf.readline().split()
            for j in range(0, 3):
                lat[i, j] = float(lv[j])
        lat_inv = 2*numpy.pi * numpy.linalg.inv(lat)
    #get atomic specifications
    elif 'ATOMIC_SPECIES' in line:
        atname = numpy.empty(ntyp, dtype=str)
        atmass = numpy.empty(ntyp, dtype=float)
        for i in range(0, ntyp):
            line = filscf.readline().split()
            atname[i] = line[0]
            atmass[i] = float(line[1])
    #get atomic positions and atom type 
    elif 'ATOMIC_POSITIONS' in line:
        at = numpy.empty(shape=(nat,4), dtype=float)
        for i in range(0, nat):
            pos = filscf.readline().split()
            
            at[i, 0] = numpy.where(atname == pos[0])[0][0]
            for j in range(1, 4):
                at[i, j] = float(pos[j])    
filscf.close()

#dynamical matrix format:
# -index1: floor(index1/3) -> atom index 1
#         index%3 -> coordinate index of corresponding atom
# -index2: floor(index2/3) -> atom index 2
#         index%3 -> coordinate index of corresponding atom    

#____________READ GM DYNMAT_____________#
dynmat = numpy.empty([nat*3, nat*3], dtype = complex)
fil_dynmat = open(dynmat_filename, 'r')

#skip the first idyn dynmats
ndyn = -1
while not ndyn == idyn:
    line = fil_dynmat.readline()
    if "Dynamical  Matrix in cartesian axes" in line:
        ndyn += 1

#read q vector in header, skip rest    
fil_dynmat.readline()
if q == None:
    q = numpy.empty(3, dtype=float)
    vec = fil_dynmat.readline().split()

    for i in range(0, 3):
        q[i] = float(vec[i + 3])
else:
    fil_dynmat.readline()
fil_dynmat.readline()

print(q)
for i in range(0, nat):
    for j in range(0, nat):
        fil_dynmat.readline()
        for k in range(0, 3):
            line = fil_dynmat.readline().split()
            for l in range(0, 3):
                number = numpy.complex(float(line[l*2]), float(line[l*2+1]))
                dynmat[i*3+k, j*3+l] = number 
fil_dynmat.close()
                
#convert to dynamical matrices by dividing with the masses
for i in range(0, nat * 3):
    if i >= 12*3:
        mass1 = atmass[0]
    else:
        mass1 = atmass[1]
    for j in range(0, nat*3):
        if j >= 12*3:
            mass2 = atmass[0]
        else:
            mass2 = atmass[1]            
        dynmat[i,j] /= numpy.sqrt(mass1 * mass2)
                
#____________IMPOSE DYNMAT HERMITICITY_____________#
for i in range(0, nat * 3):
     for j in range(0, nat * 3):
            dynmat[i,j] = 0.5 * (dynmat[i,j] + dynmat[j,i].conjugate())
            dynmat[j,i] = dynmat[i,j].conjugate() 
            
#____________IMPOSE ACOUSTIC SUM RULE_______________#
dynmat_asr = numpy.empty([nat*3,nat*3], dtype = complex)
for i in range(0, nat * 3):
    for j in range(0, nat * 3):
        if int(numpy.floor(i/3)) == int(numpy.floor(j/3)):
            asrsum = 0
            ic = j % 3  #coordinate index
            for k in range(0, nat):
                asrsum += float(dynmat[i, k*3 + ic])               
            dynmat_asr[i,j] = dynmat[i,j] - complex(asrsum,0)
        else:
            dynmat_asr[i,j] = dynmat[i,j]  

dynmat = dynmat_asr  

dynmat *= ry_to_ev
dynmat /= (aut_to_a)**2


#____________DIAGONALIZE WITH ZHEEV_____________#
#get dynmat eigenvectors
w2fh, evfh, inf = scipy.linalg.lapack.zheev(dynmat)

#convert eigenvectors from row to column vectors
evfh = evfh.transpose()  
        
nmod = evfh.shape[0]
lat_mod = numpy.copy(lat)
for i, fac in enumerate(facs):
    lat_mod[i] *= fac

#generate supercell with modulation
for imod in range(0, nmod):
    nat_mod = nat
    at_mod = numpy.empty(shape=(0,4), dtype=float)
        
    for i in range(0, facs[0]):
        for j in range(0, facs[1]):
            for k in range(0, facs[2]):
                for l, iat in enumerate(at):
                    pos = numpy.copy(iat)

                    pos[1] += i
                    pos[2] += j
                    pos[3] += k
                    
                    pos[1] /= facs[0]
                    pos[2] /= facs[1]
                    pos[3] /= facs[2]
                    
                    pos_abs = numpy.dot(lat_mod, pos[1:4])
                    q_abs = numpy.dot(lat_inv, q)
                    wave = numpy.exp(1j*numpy.dot(q_abs,pos_abs)+1j*phase)
                    disp = evfh[imod, l*3:(l+1)*3]
                    pos[1:4] += amp*(disp*wave).astype(float) / numpy.sqrt(atmass[int(pos[0])])
                
                    at_mod = numpy.vstack((at_mod, pos))
    #sort atoms
    sortby = at_mod[:,0].argsort()
    at_mod = at_mod[sortby[::1]]
       
    #write vasp file
    filmod = open(filmod_name + str(imod), 'w')
    filmod.write('\n1.0\n')
    for lv in lat_mod:
        for num in lv:
            filmod.write('\t' + str(num) + '\t')
        filmod.write('\n')
        
    nat_typ = numpy.bincount(at_mod[:,0].astype(int))
    for iname in atname:
        filmod.write(iname + '\t')
    filmod.write('\n')
    for i, n in enumerate(nat_typ):
        filmod.write(str(n) + '\t')
    filmod.write('\nDirect\n')
    for iat in at_mod:
        for i, num in enumerate(iat):
            if i == 0:
                continue
            filmod.write(str(num) + '\t')
        filmod.write('\n')
    filmod.close()




[0.5004 0.5004 0.    ]


/home/mnoe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:124: ComplexWarning: Casting complex values to real discards the imaginary part
/home/mnoe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:170: ComplexWarning: Casting complex values to real discards the imaginary part
